In [92]:
import pandas as pd
import numpy as np
import random

# Read data from CSV files
df_courses = pd.read_csv('courses.csv')
df_course = df_courses.drop_duplicates(subset=['Course Name'], keep='first')

df_student_courses = pd.read_csv('studentCourse.csv')
df_teachers = pd.read_csv('teachers.csv')
random_room_numbers = np.random.randint(301, 311, size=len(df_courses))
df_courses['Room Number'] = ['C' + str(num) for num in random_room_numbers]
# Define possible time slots
time_slots = ['9:00 AM - 10:00 AM', '1:00 PM - 4:00 AM','12:00 PM - 8:00 PM']
# Define possible days
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']

# Function to generate random paper timing considering Friday break
def generate_paper_timing(day):
    # Generate a random time slot
    paper_timing = np.random.choice(time_slots)
    
    # If it's Friday and the timing is 1-4 pm, adjust it to 9 am - 12 pm
    if day == 'Friday' and paper_timing == '1:00 PM - 4:00 PM':
        paper_timing = '2:30 AM - 5:30 PM'
    
    return paper_timing

# Function to generate random day considering Friday break
def generate_day():
    # Generate a random day
    day = np.random.choice(days)
    return day

# Assign paper timings and days to course
df_courses['Day'] = [generate_day() for _ in range(len(df_courses))]
df_courses['Paper Timing'] = [generate_paper_timing(day) for day in df_courses['Day']]
students_courses = pd.merge(df_student_courses, df_courses, on='Course Code', how='inner')
# Define rooms
rooms = df_courses['Room Number'].unique()

def generate_random_schedule():
    schedule = {}
    teacher_schedule = {teacher: None for teacher in df_teachers['Names']}  # Initialize teacher schedule
    student_exam_count = {student: 0 for student in df_student_courses['Student Name'].unique()}  # Initialize student exam count
    for index, row in students_courses.iterrows():
        student = row['Student Name']
        course_code = row['Course Code']
        course_name = row['Course Name']
        # Check if student had a consecutive exam
        if student_exam_count[student] > 0:
            prev_course_info = schedule.get((student, course_code[:-1]))
            if prev_course_info:
                prev_course_code, _ = prev_course_info  # Get previous course code
                if prev_course_code == course_code[:-1]:  # Check if previous course code is same as current course code without last character
                    continue  # Skip scheduling this course for the student

        room = random.choice(rooms)
        slot = (row['Day'], row['Paper Timing'])  # Use assigned day and paper timing from course data
        teacher = assign_teacher(slot[1], teacher_schedule, slot[0])
        if teacher:
            teacher_schedule[teacher] = slot[1]  # Update teacher's previous exam end time
        schedule[(student, course_code)] = (room, course_name, teacher, *slot)
        student_exam_count[student] += 1

    # Add break slot for all students and faculty on Friday from 1-2 PM
    friday_break_slot = ('Friday', '1:00 PM - 2:00 PM')  # Break slot on Friday from 2 PM to 3 PM
    for student in student_exam_count.keys():
        schedule[(student, 'BREAK')] = ('BREAK', 'BREAK', None, *friday_break_slot)
    faculty_break_slot = random.choice(['morning', 'afternoon'])  # Randomly choose a slot for faculty break
    for teacher in df_teachers['Names']:
        slot = ('Friday', '9:00 AM - 12:00 PM') if faculty_break_slot == 'morning' else ('Friday', '1:00 PM - 4:00 PM')  # Set break slot for faculty
        schedule[(teacher, 'BREAK')] = ('BREAK', 'BREAK', None, *slot)

    # Add a two-hour break in the week for faculty meetings
    for day in days:
        if day != 'Friday':
            for slot in ['9:00 AM - 11:00 AM', '1:00 PM - 3:00 PM']:  # Two-hour break slots
                meeting_faculty = random.sample(df_teachers['Names'].tolist(), len(df_teachers) // 2)  # Half of the faculty
                for teacher in meeting_faculty:
                    schedule[(teacher, 'MEETING')] = ('MEETING', 'MEETING', None, day, slot)

    return schedule

def assign_teacher(slot, teacher_schedule, day):
    available_teachers = [teacher for teacher, end_time in teacher_schedule.items() if not end_time or end_time <= slot[0]]
    if available_teachers:
        # Filter out teachers who are already assigned to exams at the same time on the same day
        available_teachers = [teacher for teacher in available_teachers if not teacher_schedule.get(teacher) or teacher_schedule[teacher][0] != day]
        if available_teachers:
            return random.choice(available_teachers)
    return None


def fitness_function(schedule):
    fitness = 1000  # Initialize fitness score to a positive value
    hard_constraints_fulfilled = []  # List to store fulfilled hard constraints
    soft_constraints_fulfilled = []  # List to store fulfilled soft constraints
   
    # Check teacher invigilation constraints
    teacher_schedule = {teacher: None for teacher in df_teachers['Names']}  # Initialize teacher schedule
    for (participant, course), schedule_info in schedule.items():
        room, course_name, teacher, day, time_slot = schedule_info[:5]  # Unpack the first 5 elements
        if teacher:
            if teacher_schedule.get(teacher) and teacher_schedule[teacher][0] == day:
                end_hour = 0
                if time_slot == '9:00 AM - 12:00 PM':
                    end_hour = 12
                elif time_slot == '1:00 PM - 4:00 PM':
                    end_hour = 16

                if int(teacher_schedule[teacher][1].split(':')[0]) >= end_hour:
                    fitness -= 1000  # Penalize if a teacher invigilates two exams in a row
                    hard_constraints_fulfilled.append("A teacher invigilates two exams in a row")
            teacher_schedule[teacher] = (day, time_slot)  # Update teacher's previous exam day and time slot

    # Check student consecutive exam constraint and preference for MG course before CS course
    for student in df_student_courses['Student Name'].unique():
        mg_course_exam_time = None
        for i in range(len(schedule) - 1):
            current_course, next_course = list(schedule.keys())[i:i+2]
            if current_course[0] == student and next_course[0] == student:
                current_course_code = current_course[1]
                next_course_code = next_course[1]
                if current_course_code.startswith('MG') and next_course_code.startswith('CS'):
                    mg_course_exam_time = schedule[current_course][3:5]
                elif current_course_code.startswith('CS') and next_course_code.startswith('MG'):
                    if mg_course_exam_time and mg_course_exam_time > schedule[next_course][3:5]:
                        fitness -= 100  # Penalize if CS course is scheduled before MG course for the same student
                        soft_constraints_fulfilled.append("CS course scheduled before MG course for the same student")
                elif current_course_code.startswith('CS') and next_course_code.startswith('CS'):
                    current_slot = schedule[current_course][3:5]
                    next_slot = schedule[next_course][3:5]
                    if current_slot[1] == next_slot[0]:
                        fitness -= 1000  # Penalize if a student has consecutive exams
                        soft_constraints_fulfilled.append("A student has consecutive exams")

    # Check if all hard constraints are fulfilled
    if "A teacher invigilates two exams in a row" not in hard_constraints_fulfilled:
        hard_constraints_fulfilled.append("All exams are invigilated by a teacher")

    return fitness, hard_constraints_fulfilled, soft_constraints_fulfilled


def crossover(schedule1, schedule2):
    # Perform crossover operation to create a new schedule
    #on point cross over bases on 0.5 probability
    common_keys = set(schedule1.keys()) & set(schedule2.keys())
    child_schedule = {}
    for key in common_keys:
        child_schedule[key] = schedule1[key] if random.random() < 0.5 else schedule2[key]
    return child_schedule


def mutation(schedule):
    # Perform mutation operation to introduce diversity in the population
    mutation_rate = 0.1  # 10% mutation rate
    mutated_schedule = schedule.copy()
    for key in mutated_schedule:
        if random.random() < mutation_rate:
            mutated_schedule[key] = (random.choice(rooms), *generate_paper_timing(schedule[key][0]), generate_day())
    return mutated_schedule

def genetic_algorithm(population_size, generations):
    population = [generate_random_schedule() for _ in range(population_size)]
    for generation in range(generations):
        # Evaluate fitness of each schedule
        fitness_scores = [fitness_function(schedule) for schedule in population]

        # Select parents for crossover
        parent1 = random.choices(population, weights=[max(score[0], 0) for score in fitness_scores], k=1)[0]
        parent2 = random.choices(population, weights=[max(score[0], 0) for score in fitness_scores], k=1)[0]

        # Perform crossover and mutation to create new offspring
        offspring = crossover(parent1, parent2)
        offspring = mutation(offspring)

        # Replace old population with new offspring
        population[random.randint(0, population_size - 1)] = offspring

        # Print fitness values at each iteration
        best_fitness = max(fitness_scores, key=lambda x: x[0])[0]
        print(f"Generation {generation + 1}: Best fitness = {best_fitness}")

    # Return the best schedule found
    best_schedule = max(population, key=lambda schedule: fitness_function(schedule)[0])
    return best_schedule

if __name__ == "__main__":
    # Set parameters
    population_size = 5
    generations = 3

    # Run the genetic algorithm
    best_schedule = genetic_algorithm(population_size, generations)   

ValueError: Total of weights must be greater than zero

In [62]:
schedule_list = [[key, *value] for key, value in best_schedule.items()]
print("\nBest schedule found:")
# Create a DataFrame
df = pd.DataFrame(schedule_list, columns=["Student name/Course Code", "Room", "Course Name", "Teacher", "Day", "Paper Timing"])
df.head(60)


Best schedule found:


,Student name/Course Code,Room,Course Name,Teacher,Day,Paper Timing
0,"(Sam D Edwards, AI2011)",C308,Programming for AI,Javaria Imtiaz,Monday,9:00 AM - 12:00 PM
1,"(Sarah Nolasco, AI2011)",C304,Programming for AI,Hassan Raza,Monday,9:00 AM - 12:00 PM
2,"(Muhammad Ijaz-Ul-Haq, AI2011)",C310,Programming for AI,Mehwish Hassan,Monday,9:00 AM - 12:00 PM
3,"(Muna Osman, AI2011)",C307,Programming for AI,Waseem Shahzad,Monday,9:00 AM - 12:00 PM
4,"(Hamza Bin Muhammad, AI2011)",C309,Programming for AI,Umair Arshad,Monday,9:00 AM - 12:00 PM
5,"(Sadra Keyhani, AI2011)",C309,Programming for AI,Subhan Ullah,Monday,9:00 AM - 12:00 PM
6,"(Ana Vukojevic, AI2011)",C308,Programming for AI,Hamda Khan,Monday,9:00 AM - 12:00 PM
7,"(Mohammed Z Albahar, AI2011)",C306,Programming for AI,Farah Jabeen Awan,Monday,9:00 AM - 12:00 PM
8,"(Safiya Issa, AI2011)",C304,Programming for AI,Shams Farooq,Monday,9:00 AM - 12:00 PM
9,"(Kamila Keshubay, AI2011)",C309,Programming for AI,Zainab Abaid,Monday,9:00 AM - 12:00 PM


In [7]:
best_fitness, hard_constraints_fulfilled, soft_constraints_fulfilled = fitness_function(best_schedule)
print("Hard constraints fulfilled:", hard_constraints_fulfilled)
print("Soft constraints fulfilled:", soft_constraints_fulfilled)

Hard constraints fulfilled: ['All exams are invigilated by a teacher']
Soft constraints fulfilled: []


In [35]:
x=students_courses.iloc[9]
x

Student Name       Kamila Keshubay
Course Code                 AI2011
Course Name     Programming for AI
Room Number                   C309
Day                       Thursday
Paper Timing     1:00 PM - 4:00 PM
Name: 9, dtype: object

In [36]:
y=students_courses.iloc[20]
y

Student Name         Ibrar Manawer
Course Code                 DS3011
Course Name     Big Data Analytics
Room Number                   C303
Day                         Friday
Paper Timing     2:30 AM - 5:30 PM
Name: 20, dtype: object

In [53]:



offspring ,offspring2 =  crossover1(x,y)


In [64]:
df.to_csv('students_courses1.csv', index=False)


In [80]:
df.iloc[70]

Student name/Course Code    (Eileen Haycox, CS307)
Room                                          C309
Course Name                      Computer Networks
Teacher                               Farwa Batool
Day                                         Friday
Paper Timing                    9:00 AM - 12:00 PM
Name: 70, dtype: object

In [81]:
df.iloc[71]

Student name/Course Code    (Reem N Hassan, MG220)
Room                                          C306
Course Name                   Marketing Management
Teacher                                Ayesha Bano
Day                                      Wednesday
Paper Timing                    9:00 AM - 12:00 PM
Name: 71, dtype: object